## Topic 2 - Naïve analysis of players click count (Players' Behavior Before Balancing)


As seen previously, there is a unequal distribution of articles in Wikipedia, some countries are more represented than others. But, are those countries clicked more often by players within the Wikispeedia game? Or is there other countries that are clicked more often? We will now investigate if we see a bias independent from the coutries distribution and whether the click count can be a good approximation of player's intention. 

To do so, a first naïve approach is simply to detect countries with higher click counts (see Figure 1). With this approach, it seems that players are highly biased in their way to play Wikispeedia as some countries like United States, United Kingdom, and Australia are represented by enormous dots due to their higher click count while other are almost not visible on the map.

However, a high click count can simply be due to the high number of articles associated to a particular country within the game. This does not necessarily tell us something about player's biases. Therefore, we rather focus on the ratio of click count divided by the number of articles to get a result closer to reality. On Figure 2, we see a overrepresentation of some countries like Vatican city, Brazil, or South Africa which are different from the previous ones. Therefore, part of the high click count can simply be explained by the high number of articles associated to a particular country. But there appear to be another factor influencing the click count per country as some countries remain more represented than other even when considering a scaled version of the click count.

But, can we rationally explain a differentially distributed click count? Is there other factors influcing the click count that simply player's biases? Onto the next topic to figure it out!


Plot - Interactive and fun way to represent players' behavior 
Map with dots for each country. Size of the dot corresponds to the click count per country while the color corresponds to the number of articles associated to a country. Edges drawn between dots are scaled based on the number of time that a player uses the link between two articles associated with the corresponding countries.
before and after scaling

In [184]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
import plotly.graph_objects as go
from geopy.geocoders import Nominatim
import folium

from src.utils.functions import find_pairs
from src.data.dataloader import *
from src.scripts.graphs import geolocalization, overlap_world_map_clicks_before, overlap_world_map_clicks_after


#### Import the data and do a basic preprocessing to have the data in the appropriate format

In [186]:
data = pd.read_csv("data/country_clicks_links.csv", index_col=0)
articles = data.index.tolist()
clicks = [f"{data.click_count.iloc[i]} clicks" for i in range(len(articles))]

country_clicks = data.groupby('Top_1_name')['click_count'].sum().reset_index()
top_1_counts = data['Top_1_name'].value_counts()
country_clicks['occurrences'] = country_clicks['Top_1_name'].map(top_1_counts)
country_clicks["scaled_click_count"] = country_clicks["click_count"] / country_clicks["occurrences"]

countries = country_clicks.Top_1_name.tolist()
clicks = country_clicks.click_count.tolist()
clicks_scaled = country_clicks.scaled_click_count.tolist()

finished_paths = load_path_finished_dataframe()
finished_paths_divided = finished_paths["path"].apply(lambda row: row.split(';'))

unfinished_paths = load_path_unfinished_distance_dataframe()
unfinished_paths_divided = unfinished_paths["path"].apply(lambda row: row.split(';'))

#### Compute nodes characteristics (size and color based on the country click count)

In [187]:
# Get the geolocalisation of every country
latitudes, longitudes = geolocalization(country_clicks)

# color of nodes is proportional to the click count
scaler = MinMaxScaler(feature_range=(0.25, 1.5))
normalized_counts = scaler.fit_transform([[count] for count in clicks]).flatten()
color_map = plt.cm.get_cmap('Purples')
colors_hex_before = [matplotlib.colors.to_hex(color_map(norm)) for norm in normalized_counts]

# size of nodes is proportional to the click count 
size_scaler = MinMaxScaler(feature_range=(9, 90))
node_sizes = size_scaler.fit_transform([[count] for count in clicks]).flatten()

Processing countries: 100%|██████████| 195/195 [04:08<00:00,  1.28s/country]
C:\Users\orian\AppData\Local\Temp\ipykernel_27136\3046315152.py:7: MatplotlibDeprecationWarning:

The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.



#### Identify highway paths most often used by players

In [188]:
# highway paths will define edges on our maps
# Focus on finished paths first
all_pairs_finished = find_pairs(finished_paths_divided)
all_pairs_finished_df = pd.DataFrame({'1-unit long path': all_pairs_finished})

# Then, focus on unfinished paths
all_pairs_unfinished = find_pairs(unfinished_paths_divided)
all_pairs_unfinished_df = pd.DataFrame({'1-unit long path': all_pairs_unfinished})

# Combine finished and unfinished paths
all_pairs_merged = pd.concat([all_pairs_finished_df, all_pairs_unfinished_df])

# Divide pairs into 2 columns 'From' and 'To' > Split
all_pairs_countries = all_pairs_merged.copy()
all_pairs_countries['Article_from'] = all_pairs_merged['1-unit long path'].apply(lambda row: row[0].split(',')[0] if isinstance(row[0], str) else row)
all_pairs_countries['Article_to'] = all_pairs_merged['1-unit long path'].apply(lambda row: row[1].split(',')[0] if isinstance(row[0], str) else row)

# Associate each article from columns 'From' and 'To' to their Top_1_name country
all_pairs_countries['Top1_country_From'] = all_pairs_countries['Article_from'].map(data['Top_1_name'])
all_pairs_countries['Top1_country_To'] = all_pairs_countries['Article_to'].map(data['Top_1_name'])

# Create a column with a pair of countries
all_pairs_countries['1-unit long path - COUNTRIES'] = all_pairs_countries['Top1_country_From'] + "-> " + all_pairs_countries['Top1_country_To']

# Normalize
all_pairs_countries_normalized = all_pairs_countries["1-unit long path - COUNTRIES"].value_counts() / all_pairs_countries["1-unit long path - COUNTRIES"].value_counts().sum()


#### Map 1 - Before scaling - with Folium library

In [189]:
# Create the world map of the click count per country and game path between countries before scaling
overlap_world_map_clicks_before("world_click_counts_before_scaling.html", clicks, all_pairs_countries_normalized, countries, latitudes,longitudes)

Map is saved in world_click_counts_before_scaling.html!


#### Map 2 - After scaling - with Folium library

In [190]:
# Create the world map of the click count per country and game path between countries after scaling
overlap_world_map_clicks_after("world_click_counts_after_scaling.html", clicks_scaled, all_pairs_countries_normalized, countries, latitudes,longitudes)

Map is saved in world_click_counts_after_scaling.html!
